In [1]:
import pandas as pd
import re
pd.set_option('display.max_columns', None)



In [2]:
filename = 'all_data_anonymized.csv'
df_read = pd.read_csv(filename, sep=';')
df_read.head()

C:\Users\Oscar Azrak\AppData\Local\Temp\ipykernel_19004\2576158672.py:2: DtypeWarning: Columns (15,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_read = pd.read_csv(filename, sep=';')


,todate,EQ_0,EQ_1,EQ_2,EQ_3,EQ_4,EQ_5,EQ_6,EQ_7,EQ_8,EQ_9,EQ_10,EQ_11,EQ_12,EQ_13,EQ_14,EQ_15,EQ_16,EQS_0,EQS_1,EQS_2,EQS_3,EQS_4,EQS_5,EQS_6,EQS_7,EQS_8,EQS_9,EQS_10,FI_0,FI_1,FI_2,FI_3,FI_4,FI_5,FI_6,FI_7,FI_8,FI_9,FI_10,FI_11,FI_12,FI_13,FI_14,FXD_0,FXD_1,FXD_2,FXD_3,FXD_4,FXD_5,FXD_6,FXD_7,FXD_8,FXD_9,FXE_0,FXE_1,FXE_2,FXE_3,FXE_4,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
0,1970-01-05 00:00:00,NaN,"0,004909044",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,00595314",NaN,"0,006268081","0,031211244",NaN,"-0,013558163"
1,1970-01-06 00:00:00,NaN,"-0,006821403",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,002578518",NaN,"-0,026832774","0,030467027",NaN,"0,004273504"
2,1970-01-07 00:00:00,NaN,"-0,002124071",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,000843241",NaN,"-0,021664205","0,022758218",NaN,"0,005980449"
3,1970-01-08 00:00:00,NaN,"0,000564636",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,003370124",NaN,"0,014594867","0,031951312",NaN,"-0,000914599"
4,1970-01-09 00:00:00,NaN,"-0,003075721",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"-0,005919893",NaN,"-0,013392857","-0,003133063",NaN,"-0,007552352"


In [3]:

# set date to index
dateCol = 'todate'

df_read[dateCol] = pd.to_datetime(df_read[dateCol])
df_read.set_index(dateCol, inplace=True)

# change from comma to period and convert to float

for column in df_read.columns:
    # Replace commas with periods and convert to float
    df_read[column] = df_read[column].str.replace(',', '.').astype(float)
df_read.head()

,EQ_0,EQ_1,EQ_2,EQ_3,EQ_4,EQ_5,EQ_6,EQ_7,EQ_8,EQ_9,EQ_10,EQ_11,EQ_12,EQ_13,EQ_14,EQ_15,EQ_16,EQS_0,EQS_1,EQS_2,EQS_3,EQS_4,EQS_5,EQS_6,EQS_7,EQS_8,EQS_9,EQS_10,FI_0,FI_1,FI_2,FI_3,FI_4,FI_5,FI_6,FI_7,FI_8,FI_9,FI_10,FI_11,FI_12,FI_13,FI_14,FXD_0,FXD_1,FXD_2,FXD_3,FXD_4,FXD_5,FXD_6,FXD_7,FXD_8,FXD_9,FXE_0,FXE_1,FXE_2,FXE_3,FXE_4,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
todate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1970-01-05,NaN,0.004909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005953,NaN,0.006268,0.031211,NaN,-0.013558
1970-01-06,NaN,-0.006821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002579,NaN,-0.026833,0.030467,NaN,0.004274
1970-01-07,NaN,-0.002124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000843,NaN,-0.021664,0.022758,NaN,0.005980
1970-01-08,NaN,0.000565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003370,NaN,0.014595,0.031951,NaN,-0.000915
1970-01-09,NaN,-0.003076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.005920,NaN,-0.013393,-0.003133,NaN,-0.007552


In [4]:
startdate = '1980-01-01'

df = df_read[startdate:]
df.head()

,EQ_0,EQ_1,EQ_2,EQ_3,EQ_4,EQ_5,EQ_6,EQ_7,EQ_8,EQ_9,EQ_10,EQ_11,EQ_12,EQ_13,EQ_14,EQ_15,EQ_16,EQS_0,EQS_1,EQS_2,EQS_3,EQS_4,EQS_5,EQS_6,EQS_7,EQS_8,EQS_9,EQS_10,FI_0,FI_1,FI_2,FI_3,FI_4,FI_5,FI_6,FI_7,FI_8,FI_9,FI_10,FI_11,FI_12,FI_13,FI_14,FXD_0,FXD_1,FXD_2,FXD_3,FXD_4,FXD_5,FXD_6,FXD_7,FXD_8,FXD_9,FXE_0,FXE_1,FXE_2,FXE_3,FXE_4,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
todate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1980-01-02,NaN,-0.018988,NaN,NaN,NaN,NaN,NaN,NaN,-0.018316,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.046210,NaN,0.010604,NaN,-0.005429,-0.041021,-0.021920,-0.019289
1980-01-03,NaN,-0.005294,NaN,NaN,NaN,NaN,NaN,NaN,-0.015509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.004372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.044173,NaN,0.001396,NaN,0.002396,-0.017317,-0.034213,0.001675
1980-01-04,NaN,0.012279,NaN,NaN,NaN,NaN,NaN,NaN,0.017255,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.004391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.033839,NaN,-0.014684,NaN,0.006773,0.028012,0.027159,-0.001672
1980-01-07,NaN,0.002119,NaN,NaN,NaN,NaN,NaN,NaN,0.005529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.037644,NaN,-0.021268,NaN,0.009445,-0.026609,0.013745,0.000000
1980-01-08,NaN,0.019638,NaN,NaN,NaN,NaN,NaN,NaN,0.015610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.039435,NaN,-0.015227,NaN,0.012703,0.005257,0.011143,0.000000


In [5]:
# backfill nan values
# ändra till 0 och limit 5
df.fillna(0, limit=5, inplace=True)


C:\Users\Oscar Azrak\AppData\Local\Temp\ipykernel_19004\780959912.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, limit=5, inplace=True)


In [6]:
# only keep date and columns including "EQ_" and "MACRO_"
df = df.filter(regex='EQ_|MACRO_')


df.head(20)


,EQ_0,EQ_1,EQ_2,EQ_3,EQ_4,EQ_5,EQ_6,EQ_7,EQ_8,EQ_9,EQ_10,EQ_11,EQ_12,EQ_13,EQ_14,EQ_15,EQ_16,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
todate,,,,,,,,,,,,,,,,,,,,,,,,,,
1980-01-02,0.0,-0.018988,0.0,0.0,0.0,0.0,0.0,0.0,-0.018316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046210,0.0,0.010604,0.0,-0.005429,-0.041021,-0.021920,-0.019289
1980-01-03,0.0,-0.005294,0.0,0.0,0.0,0.0,0.0,0.0,-0.015509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.044173,0.0,0.001396,0.0,0.002396,-0.017317,-0.034213,0.001675
1980-01-04,0.0,0.012279,0.0,0.0,0.0,0.0,0.0,0.0,0.017255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033839,0.0,-0.014684,0.0,0.006773,0.028012,0.027159,-0.001672
1980-01-07,0.0,0.002119,0.0,0.0,0.0,0.0,0.0,0.0,0.005529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037644,0.0,-0.021268,0.0,0.009445,-0.026609,0.013745,0.000000
1980-01-08,0.0,0.019638,0.0,0.0,0.0,0.0,0.0,0.0,0.015610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.039435,0.0,-0.015227,0.0,0.012703,0.005257,0.011143,0.000000
1980-01-09,NaN,0.000867,NaN,NaN,NaN,NaN,NaN,NaN,0.006111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.006568,NaN,-0.003696,NaN,-0.006607,0.036214,-0.024429,-0.044877
1980-01-10,NaN,0.006898,NaN,NaN,NaN,NaN,NaN,NaN,0.011597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.003304,NaN,0.009246,NaN,-0.002312,-0.013121,0.018943,-0.024100
1980-01-11,NaN,0.001048,NaN,NaN,NaN,NaN,NaN,NaN,0.005513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041458,NaN,-0.019756,NaN,0.021640,0.054973,0.019621,0.022898
1980-01-14,NaN,0.004376,NaN,NaN,NaN,NaN,NaN,NaN,0.008675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.039808,NaN,-0.010835,NaN,-0.008488,0.048594,-0.022082,0.004684


In [7]:
import numpy as np
windows = [5, 10, 20, 40, 60, 100, 180, 240, 360, 480]
window_m = [10, 30, 60, 100, 180]
# skicka in window som inoput ist

assets = df.columns


In [8]:



def add_features(df, windows):
    # Exclude 'macro' column from the feature calculation but include it in the final DataFrame
    feature_cols = [col for col in df.columns if 'macro' not in col.lower()]

    # Initialize a dictionary to hold all the new feature data
    features_dict = {}


    # Perform rolling calculations for each window size
    for w in windows:
        for col in feature_cols:
            # Create unique feature names for each statistic and window size
            features_dict[f'{col}_VaR_{w}'] = df[col].rolling(window=w, min_periods=int(w//2)).quantile(0.05)
            features_dict[f'{col}_momentum_{w}'] = df[col].rolling(window=w, min_periods=int(w//2)).sum()
            features_dict[f'{col}_avgreturn_{w}'] = df[col].rolling(window=w, min_periods=int(w//2)).mean()
            features_dict[f'{col}_skew_{w}'] = df[col].rolling(window=w, min_periods=int(w//2)).skew()
            features_dict[f'{col}_volatility_{w}'] = df[col].rolling(window=w, min_periods=int(w//2)).std()

    # Convert the dictionary of Series to a DataFrame
    features_df = pd.DataFrame(features_dict, index=df.index)

    # Identify columns containing 'MACRO' (case-insensitive)
    macro_cols = [col for col in df.columns if 'MACRO' in col.upper()]

    # Concatenate 'MACRO' columns to the features DataFrame
    if macro_cols:
        macro_df = df[macro_cols]
        features_df = pd.concat([features_df, macro_df], axis=1)

    return features_df

df = add_features(df, windows)
df.tail()




,EQ_0_VaR_5,EQ_0_momentum_5,EQ_0_avgreturn_5,EQ_0_skew_5,EQ_0_volatility_5,EQ_1_VaR_5,EQ_1_momentum_5,EQ_1_avgreturn_5,EQ_1_skew_5,EQ_1_volatility_5,EQ_2_VaR_5,EQ_2_momentum_5,EQ_2_avgreturn_5,EQ_2_skew_5,EQ_2_volatility_5,EQ_3_VaR_5,EQ_3_momentum_5,EQ_3_avgreturn_5,EQ_3_skew_5,EQ_3_volatility_5,EQ_4_VaR_5,EQ_4_momentum_5,EQ_4_avgreturn_5,EQ_4_skew_5,EQ_4_volatility_5,EQ_5_VaR_5,EQ_5_momentum_5,EQ_5_avgreturn_5,EQ_5_skew_5,EQ_5_volatility_5,EQ_6_VaR_5,EQ_6_momentum_5,EQ_6_avgreturn_5,EQ_6_skew_5,EQ_6_volatility_5,EQ_7_VaR_5,EQ_7_momentum_5,EQ_7_avgreturn_5,EQ_7_skew_5,EQ_7_volatility_5,EQ_8_VaR_5,EQ_8_momentum_5,EQ_8_avgreturn_5,EQ_8_skew_5,EQ_8_volatility_5,EQ_9_VaR_5,EQ_9_momentum_5,EQ_9_avgreturn_5,EQ_9_skew_5,EQ_9_volatility_5,EQ_10_VaR_5,EQ_10_momentum_5,EQ_10_avgreturn_5,EQ_10_skew_5,EQ_10_volatility_5,EQ_11_VaR_5,EQ_11_momentum_5,EQ_11_avgreturn_5,EQ_11_skew_5,EQ_11_volatility_5,EQ_12_VaR_5,EQ_12_momentum_5,EQ_12_avgreturn_5,EQ_12_skew_5,EQ_12_volatility_5,EQ_13_VaR_5,EQ_13_momentum_5,EQ_13_avgreturn_5,EQ_13_skew_5,EQ_13_volatility_5,EQ_14_VaR_5,EQ_14_momentum_5,EQ_14_avgreturn_5,EQ_14_skew_5,EQ_14_volatility_5,EQ_15_VaR_5,EQ_15_momentum_5,EQ_15_avgreturn_5,EQ_15_skew_5,EQ_15_volatility_5,EQ_16_VaR_5,EQ_16_momentum_5,EQ_16_avgreturn_5,EQ_16_skew_5,EQ_16_volatility_5,EQ_0_VaR_10,EQ_0_momentum_10,EQ_0_avgreturn_10,EQ_0_skew_10,EQ_0_volatility_10,EQ_1_VaR_10,EQ_1_momentum_10,EQ_1_avgreturn_10,EQ_1_skew_10,EQ_1_volatility_10,EQ_2_VaR_10,EQ_2_momentum_10,EQ_2_avgreturn_10,EQ_2_skew_10,EQ_2_volatility_10,EQ_3_VaR_10,EQ_3_momentum_10,EQ_3_avgreturn_10,EQ_3_skew_10,EQ_3_volatility_10,EQ_4_VaR_10,EQ_4_momentum_10,EQ_4_avgreturn_10,EQ_4_skew_10,EQ_4_volatility_10,EQ_5_VaR_10,EQ_5_momentum_10,EQ_5_avgreturn_10,EQ_5_skew_10,EQ_5_volatility_10,EQ_6_VaR_10,EQ_6_momentum_10,EQ_6_avgreturn_10,EQ_6_skew_10,EQ_6_volatility_10,EQ_7_VaR_10,EQ_7_momentum_10,EQ_7_avgreturn_10,EQ_7_skew_10,EQ_7_volatility_10,EQ_8_VaR_10,EQ_8_momentum_10,EQ_8_avgreturn_10,EQ_8_skew_10,EQ_8_volatility_10,EQ_9_VaR_10,EQ_9_momentum_10,EQ_9_avgreturn_10,EQ_9_skew_10,EQ_9_volatility_10,EQ_10_VaR_10,EQ_10_momentum_10,EQ_10_avgreturn_10,EQ_10_skew_10,EQ_10_volatility_10,EQ_11_VaR_10,EQ_11_momentum_10,EQ_11_avgreturn_10,EQ_11_skew_10,EQ_11_volatility_10,EQ_12_VaR_10,EQ_12_momentum_10,EQ_12_avgreturn_10,EQ_12_skew_10,EQ_12_volatility_10,EQ_13_VaR_10,EQ_13_momentum_10,EQ_13_avgreturn_10,EQ_13_skew_10,EQ_13_volatility_10,EQ_14_VaR_10,EQ_14_momentum_10,EQ_14_avgreturn_10,EQ_14_skew_10,EQ_14_volatility_10,EQ_15_VaR_10,EQ_15_momentum_10,EQ_15_avgreturn_10,EQ_15_skew_10,EQ_15_volatility_10,EQ_16_VaR_10,EQ_16_momentum_10,EQ_16_avgreturn_10,EQ_16_skew_10,EQ_16_volatility_10,EQ_0_VaR_20,EQ_0_momentum_20,EQ_0_avgreturn_20,EQ_0_skew_20,EQ_0_volatility_20,EQ_1_VaR_20,EQ_1_momentum_20,EQ_1_avgreturn_20,EQ_1_skew_20,EQ_1_volatility_20,EQ_2_VaR_20,EQ_2_momentum_20,EQ_2_avgreturn_20,EQ_2_skew_20,EQ_2_volatility_20,EQ_3_VaR_20,EQ_3_momentum_20,EQ_3_avgreturn_20,EQ_3_skew_20,EQ_3_volatility_20,EQ_4_VaR_20,EQ_4_momentum_20,EQ_4_avgreturn_20,EQ_4_skew_20,EQ_4_volatility_20,EQ_5_VaR_20,EQ_5_momentum_20,EQ_5_avgreturn_20,EQ_5_skew_20,EQ_5_volatility_20,EQ_6_VaR_20,EQ_6_momentum_20,EQ_6_avgreturn_20,EQ_6_skew_20,EQ_6_volatility_20,EQ_7_VaR_20,EQ_7_momentum_20,EQ_7_avgreturn_20,EQ_7_skew_20,EQ_7_volatility_20,EQ_8_VaR_20,EQ_8_momentum_20,EQ_8_avgreturn_20,EQ_8_skew_20,EQ_8_volatility_20,EQ_9_VaR_20,EQ_9_momentum_20,EQ_9_avgreturn_20,EQ_9_skew_20,EQ_9_volatility_20,EQ_10_VaR_20,EQ_10_momentum_20,EQ_10_avgreturn_20,EQ_10_skew_20,EQ_10_volatility_20,EQ_11_VaR_20,EQ_11_momentum_20,EQ_11_avgreturn_20,EQ_11_skew_20,EQ_11_volatility_20,EQ_12_VaR_20,EQ_12_momentum_20,EQ_12_avgreturn_20,EQ_12_skew_20,EQ_12_volatility_20,EQ_13_VaR_20,EQ_13_momentum_20,EQ_13_avgreturn_20,EQ_13_skew_20,EQ_13_volatility_20,EQ_14_VaR_20,EQ_14_momentum_20,EQ_14_avgreturn_20,EQ_14_skew_20,EQ_14_volatility_20,EQ_15_VaR_20,EQ_15_momentum_20,EQ_15_avgreturn_20,EQ_15_skew_20,EQ_15_volatility_20,EQ_16_VaR_20,EQ_16_momentum_20,EQ_

In [9]:
assets


Index(['EQ_0', 'EQ_1', 'EQ_2', 'EQ_3', 'EQ_4', 'EQ_5', 'EQ_6', 'EQ_7', 'EQ_8',
       'EQ_9', 'EQ_10', 'EQ_11', 'EQ_12', 'EQ_13', 'EQ_14', 'EQ_15', 'EQ_16',
       'MACRO_8', 'MACRO_0', 'MACRO_1', 'MACRO_2', 'MACRO_3', 'MACRO_4',
       'MACRO_5', 'MACRO_6', 'MACRO_7'],
      dtype='object')

In [10]:
df.tail(50)

EQ_0_VaR_5  EQ_0_momentum_5  EQ_0_avgreturn_5  EQ_0_skew_5  \
todate                                                                   
2023-11-15   -0.007128         0.024318          0.004864    -1.159096   
2023-11-16   -0.009030         0.007262          0.001452     0.102730   
2023-11-17   -0.004277         0.025947          0.005189    -0.734770   
2023-11-20   -0.004570         0.021938          0.004388    -0.206087   
2023-11-21   -0.005453         0.005401          0.001080     0.146106   
2023-11-22   -0.005453         0.006278          0.001256     0.028968   
2023-11-23   -0.001669         0.014517          0.002903     0.334205   
2023-11-24   -0.001669         0.007837          0.001567    -1.384557   
2023-11-27   -0.003584         0.002133          0.000427    -0.406123   
2023-11-28   -0.003528         0.002414          0.000483    -0.437702   
2023-11-29   -0.003528         0.000481          0.000096    -0.765361   
2023-11-30   -0.003528         0.004041          0.000808    -0.089984   
2023-12-01   -0.003528         0.006552          0.001310    -0.336497   
2023-12-04   -0.002021         0.009373          0.001875    -0.184824   
2023-12-05   -0.000363         0.018971          0.003794    -0.776398   
2023-12-06    0.000138         0.023303          0.004661    -1.778244   
2023-12-07   -0.001279         0.016339          0.003268    -0.399806   
2023-12-08   -0.001279         0.024460          0.004892     0.155113   
2023-12-11   -0.000412         0.028795          0.005759    -0.014831   
2023-12-12   -0.001300         0.020345          0.004069     0.804789   
2023-12-13   -0.001696         0.011958          0.002392     1.713567   
2023-12-14   -0.001652         0.018844          0.003769     0.946309   
2023-12-15   -0.001652         0.008708          0.001742    -0.104532   
2023-12-18   -0.003249         0.001779          0.000356     0.631435   
2023-12-19   -0.003249         0.003691          0.000738     0.180017   
2023-12-20   -0.002734         0.006462          0.001292    -0.429914   
2023-12-21   -0.003366        -0.001448         -0.000290    -0.325919   
2023-12-22   -0.003366        -0.004340         -0.000868    -0.634082   
2023-12-25   -0.002033        -0.000725         -0.000181    -1.460083   
2023-12-26   -0.002145        -0.001514         -0.000505    -0.935641   
2023-12-27   -0.002145        -0.002435         -0.000812    -1.690514   
2023-12-28   -0.005068        -0.005682         -0.001894    -1.724734   
2023-12-29   -0.005048        -0.004358         -0.001453    -1.548372   
2024-01-01   -0.005048        -0.004358         -0.001453    -1.548372   
2024-01-02   -0.004953        -0.005550         -0.001388    -1.391383   
2024-01-03   -0.014640        -0.021849         -0.005462    -1.273269   
2024-01-04   -0.013976        -0.011127         -0.002782    -1.537382   
2024-01-05   -0.014422        -0.016482         -0.004120    -0.898253   
2024-01-08   -0.013818        -0.012721         -0.002544    -1.260460   
2024-01-09   -0.013818        -0.014819         -0.002964    -1.029076   
2024-01-10   -0.003988         0.001144          0.000229     0.187280   
2024-01-12   -0.004032        -0.003961         -0.000990     0.923064   
2024-01-15   -0.006603        -0.006986         -0.001747     0.036455   
2024-01-16   -0.006603        -0.012698         -0.003175    -0.997239   
2024-01-17   -0.010171        -0.020106         -0.005026    -0.337335   
2024-01-18   -0.010171        -0.009072         -0.002268     1.199519   
2024-01-19   -0.009996        -0.013323         -0.002665     1.370465   
2024-01-22   -0.009408        -0.000538         -0.000108     0.143766   
2024-01-23   -0.009408        -0.002022         -0.000404     0.292081   
2024-01-24   -0.004088         0.017677          0.003535    -0.317899   

            EQ_0_volatility_5  EQ_1_VaR_5  EQ_1_momentum_5  EQ_1_avgreturn_5  \
todate                                                                         
202

In [11]:
# Reset the index to make it a regular column
df_reset = df.reset_index()

# Now, you can specify the former index column (now a regular column) as the id_vars
long_df = df_reset.melt(id_vars='todate', var_name='metric', value_name='value')



In [12]:
def parse_metric(row):
    parts = row['metric'].split('_')
    row['asset'] = parts[1]  # For other metrics, the second part is the asset
    row['metric_type'] = '_'.join(parts[2:])  # The rest is the metric type
    return row

In [14]:
long_df.tail(50) 

,todate,metric,value
9869860,2023-11-15,MACRO_7,-0.020083
9869861,2023-11-16,MACRO_7,-0.012531
9869862,2023-11-17,MACRO_7,-0.004947
9869863,2023-11-20,MACRO_7,-0.013185
9869864,2023-11-21,MACRO_7,0.021246
9869865,2023-11-22,MACRO_7,0.001287
9869866,2023-11-23,MACRO_7,0.000000
9869867,2023-11-24,MACRO_7,-0.012531
9869868,2023-11-27,MACRO_7,-0.026464
9869869,2023-11-28,MACRO_7,0.019608


In [19]:

# Use vectorized string operations to split 'metric' column and expand to new DataFrame
split_metrics = long_df['metric'].str.split('_', expand=True)

# Determine which rows correspond to 'MACRO' metrics
macro_mask = split_metrics[0] == 'MACRO'

# For non-'MACRO' metrics, construct 'asset' by concatenating the first two parts, and the rest as 'metric_type'
long_df['asset'] = split_metrics[0] + '_' + split_metrics[1]
long_df['metric_type'] = split_metrics[2]+ '_' + split_metrics[3]

# Reset index on the left-hand side DataFrame slice to ensure alignment
lhs = long_df.loc[macro_mask, 'metric_type'].reset_index(drop=True)

# Reset index on the right-hand side Series to ensure alignment
rhs = (split_metrics.loc[macro_mask, 0] + '_' + split_metrics.loc[macro_mask, 1]).reset_index(drop=True)

# Assign the values after ensuring both sides have the same length
lhs = rhs

# Assign the modified Series back to the original DataFrame (if needed)
long_df.loc[macro_mask, 'metric_type'] = lhs.values

# For 'MACRO' metrics, set 'metric_type' directly as the metric name, and use 'MACRO' as a placeholder in 'asset'
long_df.loc[macro_mask, 'metric_type'] = split_metrics.loc[macro_mask, 0] + '_' + split_metrics.loc[macro_mask, 1]
long_df.loc[macro_mask, 'asset'] = 'MACRO'

# Remove 'MACRO' placeholder rows to avoid 'MACRO' appearing in the 'asset' column of the final DataFrame
long_df = long_df[long_df['asset'] != 'MACRO']

# Pivot the DataFrame to wide format, with 'date' and 'asset' as indexes, and 'metric_type' as columns
final_df = long_df.pivot_table(index=['todate', 'asset'], columns='metric_type', values='value').reset_index()

# Incorporate 'MACRO' metrics as separate columns
# Since 'MACRO' metrics do not depend on 'asset', we can directly merge them on 'todate'
macro_df = df.filter(regex='^MACRO').copy()
macro_df['todate'] = df_reset['todate']


In [25]:
if 'todate' in macro_df.columns:
    macro_df = macro_df.drop(columns='todate')

# Reset the index to turn the 'todate' index into a column
macro_df = macro_df.reset_index()
macro_df.head()

,todate,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
0,1980-01-02,0.0,0.046210,0.0,0.010604,0.0,-0.005429,-0.041021,-0.021920,-0.019289
1,1980-01-03,0.0,0.044173,0.0,0.001396,0.0,0.002396,-0.017317,-0.034213,0.001675
2,1980-01-04,0.0,0.033839,0.0,-0.014684,0.0,0.006773,0.028012,0.027159,-0.001672
3,1980-01-07,0.0,0.037644,0.0,-0.021268,0.0,0.009445,-0.026609,0.013745,0.000000
4,1980-01-08,0.0,-0.039435,0.0,-0.015227,0.0,0.012703,0.005257,0.011143,0.000000


In [27]:




final_df = pd.merge(final_df, macro_df, on='todate', how='left')

# Rename 'todate' to 'date' for consistency with your request
final_df.rename(columns={'todate': 'date'}, inplace=True)


KeyError: 'todate'

In [28]:

# Display the first few rows of the transformed DataFrame
final_df.tail()

,date,asset,VaR_10,VaR_100,VaR_180,VaR_20,VaR_240,VaR_360,VaR_40,VaR_480,VaR_5,VaR_60,avgreturn_10,avgreturn_100,avgreturn_180,avgreturn_20,avgreturn_240,avgreturn_360,avgreturn_40,avgreturn_480,avgreturn_5,avgreturn_60,momentum_10,momentum_100,momentum_180,momentum_20,momentum_240,momentum_360,momentum_40,momentum_480,momentum_5,momentum_60,skew_10,skew_100,skew_180,skew_20,skew_240,skew_360,skew_40,skew_480,skew_5,skew_60,volatility_10,volatility_100,volatility_180,volatility_20,volatility_240,volatility_360,volatility_40,volatility_480,volatility_5,volatility_60,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
152438,2024-01-24,EQ_5,-0.003774,-0.018025,-0.017813,-0.012354,-0.017515,-0.019780,-0.012008,-0.024038,0.001749,-0.010077,0.005562,0.001069,0.001350,0.001970,0.001423,0.000918,0.002270,0.000253,0.008714,0.003342,0.044498,0.101561,0.230864,0.033491,0.328783,0.317722,0.081717,0.116651,0.043569,0.183825,0.522397,-0.301178,-0.188461,0.180269,-0.018147,0.176234,-0.141807,-0.148178,0.663112,0.139199,0.007909,0.010040,0.010341,0.009777,0.010442,0.013806,0.008513,0.015836,0.007527,0.008780,0.077863,-0.004837,0.009681,0.004011,0.00048,0.032585,0.029895,-0.018139,0.023889
152439,2024-01-24,EQ_6,-0.009403,-0.012402,-0.012146,-0.006308,-0.012826,-0.013320,-0.012117,-0.015090,-0.000079,-0.008856,0.000166,0.000322,-0.000032,0.000337,-0.000034,0.000303,0.001210,-0.000077,0.003046,0.002005,0.001495,0.030600,-0.005537,0.006071,-0.007878,0.104457,0.043549,-0.035298,0.015229,0.112304,-1.204467,0.327924,0.155982,-0.639823,0.003741,0.371582,-0.209177,0.044854,0.402613,0.995721,0.005842,0.008073,0.007581,0.005120,0.007494,0.008532,0.006521,0.008956,0.003237,0.007468,0.077863,-0.004837,0.009681,0.004011,0.00048,0.032585,0.029895,-0.018139,0.023889
152440,2024-01-24,EQ_7,-0.010991,-0.011553,-0.012686,-0.013519,-0.013934,-0.016039,-0.011149,-0.017247,-0.003090,-0.010791,-0.000349,0.000514,0.000143,-0.001143,0.000202,0.000578,0.001330,0.000183,0.004875,0.001982,-0.003141,0.049339,0.024689,-0.020574,0.046490,0.201125,0.047878,0.084915,0.024373,0.111005,0.322725,0.375054,0.274079,-0.087644,-0.271348,0.099097,-0.151393,0.125507,0.143475,0.109353,0.008473,0.008872,0.008588,0.007911,0.009585,0.010545,0.007699,0.011367,0.006935,0.008209,0.077863,-0.004837,0.009681,0.004011,0.00048,0.032585,0.029895,-0.018139,0.023889
152441,2024-01-24,EQ_8,-0.010982,-0.016741,-0.016408,-0.015310,-0.018345,-0.022380,-0.015143,-0.025436,-0.007669,-0.016065,0.000609,0.000450,0.000653,-0.001690,0.000084,0.000177,0.002763,-0.000192,0.004845,0.003158,0.004873,0.042792,0.111610,-0.028731,0.019389,0.061342,0.099464,-0.088405,0.024224,0.173717,0.804852,0.871420,0.730038,0.082732,0.383894,0.330117,0.307668,0.031820,0.338015,0.855891,0.011094,0.013419,0.012384,0.011610,0.012575,0.014043,0.013522,0.015047,0.011586,0.014788,0.077863,-0.004837,0.009681,0.004011,0.00048,0.032585,0.029895,-0.018139,0.023889
152442,2024-01-24,EQ_9,-0.009556,-0.009369,-0.010586,-0.008702,-0.010629,-0.010954,-0.005399,-0.011819,-0.008832,-0.006173,-0.000359,0.000190,-0.000202,0.000316,0.000032,0.000131,0.001118,-0.000124,0.000829,0.001383,-0.003233,0.018015,-0.034686,0.005374,0.007335,0.045479,0.039121,-0.057077,0.004146,0.076059,-0.329389,-0.318762,-0.140729,-0.383149,-0.105974,0.110845,-0.440185,-0.034869,-0.791481,-0.009283,0.006161,0.006186,0.006565,0.005308,0.006967,0.007502,0.004551,0.007955,0.007543,0.004941,0.077863,-0.004837,0.009681,0.004011,0.00048,0.032585,0.029895,-0.018139,0.023889


Index(['VaR', 'momentum', 'avgreturn', 'skew', 'volatility', nan], dtype='object')
Index(['VaR', 'momentum', 'avgreturn', 'skew', 'volatility'], dtype='object')


In [13]:
"""def condstruct_df(df, feature_names, windows, macro_columns):
    column_names = ['Date', 'Asset']
    for feature_name in feature_names:
        for window in windows:
            column_names.extend([f'{feature_name}_{window}'])

    column_names.extend(macro_columns)
    final_rows = []

    for date in df.index:
        for asset in assets:
            row = [date, asset]
            for feature_name in feature_names:
                # Filter the DataFrame for the current feature
                feature_df = df.filter(regex=f'^{asset}_{feature_name}_\\d+')
                
                for window in windows:
                    # Construct the regex pattern for this feature and window
                    pattern = f'^{asset}_{feature_name}_{window}$'
                    
                    # Find the column that matches this pattern
                    matching_columns = [col for col in feature_df.columns if re.fullmatch(pattern, col)]
                    
                    # If a matching column is found, retrieve the value, otherwise use NaN
                    value = feature_df.loc[date, matching_columns[0]] if matching_columns else float('nan')
                    
                    row.append(value)
                
            for macro_col in macro_columns:
                # Directly copy the value from the macro column
                macro_col_full_name = f'{macro_col}'  # Adjust according to your naming convention
                if macro_col_full_name in df.columns:
                    value = df.loc[date, macro_col_full_name]
                else:
                    value = float('nan')
                row.append(value)

            
            final_rows.append(row)
    # Create the final DataFrame
    final_df = pd.DataFrame(final_rows, columns=column_names)
    return final_df"""

import pandas as pd
import re

def condstruct_df(df, feature_names, windows, macro_columns, assets):
    # Initialize column names with 'Date' and 'Asset'
    column_names = ['Date', 'Asset']

    # exclude values from macro_columns from assets list
    assets = [asset for asset in assets if asset not in macro_columns]
    
    # Extend column names with feature-based columns for each asset
    for feature_name in feature_names:
        for window in windows:
            column_names.append(f'{feature_name}_{window}')
    
    # Extend column names with macro columns
    column_names.extend(macro_columns)
    
    # List to store all rows before creating the final DataFrame
    final_rows = []

    # Iterate over each date in the DataFrame's index
    for date in df.index:
        # Retrieve macro values for this date to avoid repetition
        macro_values_for_date = [df.loc[date, macro_col] if macro_col in df.columns else float('nan') for macro_col in macro_columns]
        
        # Iterate over each asset
        for asset in assets:
            # Start constructing the row with 'Date' and 'Asset'
            row = [date, asset]
            
            # Iterate over each feature name and window
            for feature_name in feature_names:
                feature_df = df.filter(regex=f'^{asset}_{feature_name}_\\d+')
                
                for window in windows:
                    pattern = f'^{asset}_{feature_name}_{window}$'
                    matching_columns = [col for col in feature_df.columns if re.fullmatch(pattern, col)]
                    value = feature_df.loc[date, matching_columns[0]] if matching_columns else float('nan')
                    row.append(value)
            
            # Extend the row with macroeconomic values for this date
            row.extend(macro_values_for_date)
            
            # Add the completed row to the final list of rows
            final_rows.append(row)

    # Create the final DataFrame with all rows and column names
    final_df = pd.DataFrame(final_rows, columns=column_names)
    
    return final_df

    

,Date,Asset,VaR_5,VaR_10,VaR_20,VaR_40,VaR_60,VaR_100,VaR_180,VaR_240,VaR_360,VaR_480,momentum_5,momentum_10,momentum_20,momentum_40,momentum_60,momentum_100,momentum_180,momentum_240,momentum_360,momentum_480,avgreturn_5,avgreturn_10,avgreturn_20,avgreturn_40,avgreturn_60,avgreturn_100,avgreturn_180,avgreturn_240,avgreturn_360,avgreturn_480,skew_5,skew_10,skew_20,skew_40,skew_60,skew_100,skew_180,skew_240,skew_360,skew_480,volatility_5,volatility_10,volatility_20,volatility_40,volatility_60,volatility_100,volatility_180,volatility_240,volatility_360,volatility_480,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
0,1980-01-02,EQ_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.04621,0.0,0.010604,0.0,-0.005429,-0.041021,-0.02192,-0.019289
1,1980-01-02,EQ_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.04621,0.0,0.010604,0.0,-0.005429,-0.041021,-0.02192,-0.019289
2,1980-01-02,EQ_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.04621,0.0,0.010604,0.0,-0.005429,-0.041021,-0.02192,-0.019289
3,1980-01-02,EQ_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.04621,0.0,0.010604,0.0,-0.005429,-0.041021,-0.02192,-0.019289
4,1980-01-02,EQ_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.04621,0.0,0.010604,0.0,-0.005429,-0.041021,-0.02192,-0.019289
5,1980-01-02,EQ_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.04621,0.0,0.010604,0.0,-0.005429,-0.041021,-0.02192,-0.019289
6,1980-01-02,EQ_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.04621,0.0,0.010604,0.0,-0.005429,-0.041021,-0.02192,-0.019289
7,1980-01-02,EQ_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.04621,0.0,0.010604,0.0,-0.005429,-0.041021,-0.02192,-0.019289
8,1980-01-02,EQ_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.04621,0.0,0.010604,0.0,-0.005429,-0.041021,-0.02192,-0.019289
9,1980-01-02,EQ_9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.04621,0.0,0.010604,0.0,-0.005429,-0.041021,-0.02192,-0.019289


,Date,Asset,VaR_5,VaR_10,VaR_20,VaR_40,VaR_60,VaR_100,VaR_180,VaR_240,VaR_360,VaR_480,momentum_5,momentum_10,momentum_20,momentum_40,momentum_60,momentum_100,momentum_180,momentum_240,momentum_360,momentum_480,avgreturn_5,avgreturn_10,avgreturn_20,avgreturn_40,avgreturn_60,avgreturn_100,avgreturn_180,avgreturn_240,avgreturn_360,avgreturn_480,skew_5,skew_10,skew_20,skew_40,skew_60,skew_100,skew_180,skew_240,skew_360,skew_480,volatility_5,volatility_10,volatility_20,volatility_40,volatility_60,volatility_100,volatility_180,volatility_240,volatility_360,volatility_480,MACRO_8,MACRO_0,MACRO_1,MACRO_2,MACRO_3,MACRO_4,MACRO_5,MACRO_6,MACRO_7
298690,2024-01-23,EQ_2,-0.007424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.155895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012887,0.001778,-0.005217,0.005032,0.034542,0.015115,0.010638,0.003641,0.000000
298691,2024-01-23,EQ_3,-0.036424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.036237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.288971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.024380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012887,0.001778,-0.005217,0.005032,0.034542,0.015115,0.010638,0.003641,0.000000
298692,2024-01-23,EQ_4,-0.011528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.565225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012887,0.001778,-0.005217,0.005032,0.034542,0.015115,0.010638,0.003641,0.000000
298693,2024-01-23,EQ_5,-0.004346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006549,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.150206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012887,0.001778,-0.005217,0.005032,0.034542,0.015115,0.010638,0.003641,0.000000
298694,2024-01-23,EQ_6,-0.009739,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.003153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.566513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012887,0.001778,-0.005217,0.005032,0.034542,0.015115,0.010638,0.003641,0.000000
298695,2024-01-23,EQ_7,-0.011296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.083067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012887,0.001778,-0.005217,0.005032,0.034542,0.015115,0.010638,0.003641,0.000000
298696,2024-01-23,EQ_8,-0.006933,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.025144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.417538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.011321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012887,0.001778,-0.005217,0.005032,0.034542,0.015115,0.010638,0.003641,0.000000
298697,2024-01-23,EQ_9,-0.009983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.007101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.262913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012887,0.001778,-0.005217,0.005032,0.034542,0.015115,0.010638,0.003641,0.000000
298698,2024-01-23,EQ_10,-0.007242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.005902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.001180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.972494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012887,0.001778,-0.005217,0.005032,0.034542,0.015115,0.010638,0.003641,0.000000
298699,2024-01-23,EQ_11,-0.002949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.002811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.644529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.007367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.012887,0.001778,-0.005217,0.005032,0.034542,0.015115,0.010638,0.003641,0.000000


In [30]:
# save the df as pickle in pickles folder

import pickle

with open('final_df.pkl', 'wb') as f:
    pickle.dump(final_df, f)
    f.close()

